In [ ]:
import Pkg
Pkg.activate(".")

pkgs = [
"ArgParse",
"Base64",
"BioSequences",
"DataFrames",
"Dates",
"DelimitedFiles",
"FASTX",
"GLM",
"HTTP",
"JSON",
"Graphs",
"MetaGraphs",
"MD5",
"Statistics",
"StatsPlots",
"uCSV",
"CodecZlib",
"YAML",
"Revise",
"Kmers",
"StatsBase"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

import Mycelia

In [ ]:
# mamba create -n mmseqs2 -c bioconda mmseqs2

# conda run --no-capture-output -n mmseqs2 mmseqs 

In [ ]:
# MMSeqs Easy taxonomy
# amino acid
# ~ 5 hour download time may not be tolerable?
# processing
# 7930.350543 seconds (710.71 k allocations: 40.309 MiB, 0.00% gc time, 0.00% compilation time)
# 2 hours 12m
# 1.5 hours for iterative search with varying sensitivity
# Mycelia.download_mmseqs_db(db="UniRef100", conda_env="viral-pangenome-discovery")
# --force-reuse 1
# --remove-tmp-files 1
# conda run --live-stream -n viral-pangenome-discovery

# download on NERSC is way too slow - going to skip for now but will consider retrying later
# mkdir -p $HOME/workspace/mmseqs
# conda run --no-capture-output -n mmseqs2 mmseqs databases --compressed 1 UniRef100 $HOME/workspace/mmseqs/UniRef100 $HOME/workspace/mmseqs/tmp
# conda run --no-capture-output -n mmseqs2 mmseqs databases --compressed 1 UniRef90 $HOME/workspace/mmseqs/UniRef90 $HOME/workspace/mmseqs/tmp
# conda run --no-capture-output -n mmseqs2 mmseqs databases --compressed 1 UniRef50 $HOME/workspace/mmseqs/UniRef50 $HOME/workspace/mmseqs/tmp

In [ ]:
data_dir = joinpath(dirname(pwd()), "data")

In [ ]:
SRR_paths = filter(x -> !occursin(".ipynb_checkpoints", x), readdir(joinpath(data_dir, "SRA"), join=true))
SRR_paths = filter(x -> isfile(joinpath(x, "megahit", "final.contigs.fastg.gfa.fna")), SRR_paths)
# filter out samples that have already been processed
# SRR_paths = filter(x -> !isfile(joinpath(x, basename(x) * ".contig_info.tsv")), SRR_paths)

In [ ]:
target_database = 
for target_database in [
    "$(homedir())/workspace/mmseqs/UniRef50",
    "$(homedir())/workspace/mmseqs/UniRef90",
    "$(homedir())/workspace/mmseqs/UniRef100",
]
    for SRR_path in SRR_paths
        SRR = basename(SRR_path)
        assembled_fasta = joinpath(SRR_path, "megahit", "final.contigs.fastg.gfa.fna")
        out_dir = mkpath(joinpath(SRR_path, "mmseqs_easy_taxonomy"))
        outfile_base = joinpath(out_dir, basename(assembled_fasta) * ".mmseqs_easy_taxonomy." * basename(target_database))
        outfiles = [outfile_base * ext for ext in ["_lca.tsv", "_report", "_tophit_aln", "_tophit_report"]]
        if !all(isfile, outfiles)
            println("need to run $target_database for $SRR")
            # run(`sbatch mmseqs.sh $(assembled_fasta) $(target_database) $(outfile_base) $(joinpath(out_dir, "tmp"))`)
            # println(`sbatch mmseqs.sh $(assembled_fasta) $(target_database) $(outfile_base) $(joinpath(out_dir, "tmp"))`)
            # 73 minutes UniRef100
            # 35 minutes UniRef90
            # 10 minutes UniRef50
            # println("conda run --no-capture-output -n mmseqs2 mmseqs easy-taxonomy $(assembled_fasta) $(target_database) $(outfile_base) $(joinpath(out_dir, "tmp"))")
            # Mycelia.run_mmseqs_easy_taxonomy(out_dir=SRR_path, query_fasta=assembled_fasta, target_database=target_database, outfile=SRR)
        else
            println("all outfiles already exists for $target_database and $SRR...")
        end
    end
end

In [ ]:
# python $HOME/workspace/Mycelia/projects/viral-pangenome-discovery/notebooks/kreport2krona.py -r $HOME/workspace/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399459/mmseqs_easy_taxonomy_default/SRR6399459.mmseqs_easy_taxonomy.UniRef100.txt_report -o $HOME/workspace/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399459/mmseqs_easy_taxonomy_default/SRR6399459.mmseqs_easy_taxonomy.UniRef100.txt_report.krona
# mamba install -c bioconda krona
# ktUpdateTaxonomy.sh
# ktImportText $HOME/workspace/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399459/mmseqs_easy_taxonomy_default/SRR6399459.mmseqs_easy_taxonomy.UniRef100.txt_report.krona -o $HOME/workspace/Mycelia/projects/viral-pangenome-discovery/data/SRA/SRR6399459/mmseqs_easy_taxonomy_default/SRR6399459.mmseqs_easy_taxonomy.UniRef100.txt_report.krona.html
# if !isfile(krona_file)
#     run(`python kreport2krona.py -r $(report) -o $(krona_file)`)
# end
# if !isfile(krona_html)
#     run(`ktImportText $(krona_file) -o $(krona_html)`)
# end